In [ ]:
# Imports 
import openpyxl
import os

In [ ]:
os.getcwd()

In [ ]:
# Define filepaths (duration - 1 min)

excel_name = "Coo_Costs_2016-2023.xlsx"
excel_path = os.path.join(os.getcwd(), excel_name)

workbook = openpyxl.load_workbook(excel_path)

# Select the source sheet and the results sheet
source_sheet = workbook["Gesamtkosten"]         # Excel sheet aus dem die Daten genommen werden
result_sheet = workbook.create_sheet("result")               # Excel sheet in den die Daten einzufügen sind
end_result_sheet = workbook.create_sheet("end result")     

In [ ]:
#
# Create the search parameters (duration - 10 sec)
#

# Initialize a set for search parameters in the first column

parameter_column1 = set()                            # first column (= Liste aller Kostenartenbezeichnungen); dazu benötigen wir ein set(), welches werte nur in einfacher Ausfertigung aufnimmt (= keine Mehrfachnennungen innerhalb des sets, sonst würden die Werte doppelt berücksichtigt)
# Iterate over the rows in column B to collect unique search parameters
for cell_b in source_sheet['B'][1:]:
    if cell_b.value is not None:
        parameter_column1.add(cell_b.value)

# Initialize a list for search parameters in the first column               
parameter_column2 = ["2016","2017", "2018", "2019", "2020", "2021", "2022", "2023"]         # second column (= Geschäftsjahre)


print(parameter_column1)
print(parameter_column2)

In [ ]:
# Extract data from the excel sheets (duration - 2 min. )

# Initiate an index for the columns so each year can be stored separate columns

col_index = 1

# initialize variables for the P&L cost centres

material_costs = 0
manufacturing_costs = 0
engineering_project_management_costs = 0
imputed_costs = 0
installation_erection_costs = 0
other_costs = 0
overhead_costs = 0

material_cost_parameters = {"Bestimmungsmaterial", "Lagerentnahmen"}   

manufacturing_cost_parameters = {"Eigene Konstr. - Pl", "Eigene Konstruktion", "Fertigungsmat. - Pl.", "Konzernzulieferung"}

engineering_project_management_cost_parameters = {"Bez. Konstr-leist Dr", "Bez. Konstr-leist vU", "Eig. Fert. AV", 
"Eig. Fert. Hand/Mont", "Eig. Fert. Konstr.", "Eig. Fert. Qualität","Kaufm.Projektleitung" ,"Projektleit. Kaufm.", 
"Projektleitung Konst" ,"sonstige Dienstleist", "sonst.bez.Leist. Dri"} 

imputed_cost_parameters = {"Auftragszinsen (PoC)", "Kalk. Lizenzen", "Kalk. Lizenzen (POC)", "Kalk. Wagnis OWN-Mat", "Kalk. Wagnis OWN-LV", 
"Kalk. Wagnis OWN-Mat", "Kalk. Zinsen intrag.", "Kalk. Zinsen OWN-LV", "Kalk. Zinsen OWN-Mat", "Kalk.Wag.Intra Group"} 

installation_erection_cost_parameters = {"Bez. Inbetrn-leist D", "Bez. Inbetrn-leist v", "Bez. Montageleist.Dr", 
"Eigene Montage / IBN", "Fremdpersonal Fertig", "Sonst.Fremdleist.Dri"}  

other_cost_parameters = {"and.vertriebsbed.Auf", "Anlagenserv.(v.verb.", "Ausg.-frachten", "Avalprovisionen", "Bewirtungskost-abzf", 
"Bewirtungskost-nabzf", "Büromaterial", "Eingangszölle", "Eing-u.Zwischenfrach", "Fahrdienst", "Fertigungssonderkost", 
"Garantieversicherung", "Monteurauslösungen", "Nebenkosten Geldverk", "Porto und Ähnliches", "Postdienste vU", "Reisekost. akt.-fäh.", 
"Reisekost. allg.", "Reisekost. n.akt.-fä", "Sonst.Beratungskoste", "sonstige Gebühren/Be", "Sonstige Mieten/Pach", "Sonstige Vertriebsko", 
"Transportversicherun", "Übersetzungsarb. Dri", "Verpackung abw. Meng", "Verpackungsmaterial", "Verpflegungspauschal", 
"Wartung/Rep. v. BuGA", "Werbegeschenke-abzf"}  

overhead_cost_parameters = {"MGK Intra Group-Mat", "MGK OWN-Bestimm.Mat.", "MGK OWN-Lagermat.", "VTGK Intra Group-Mat", 
"VTGK OWN-Lagermat.", "VTGK OWN-LV", "VV-Zuschlag ALED", "VWGK Intra Group-Mat", "VWGK OWN-Lagermat.", "VWGK OWN-LV"
}   

cost_parameters = [material_cost_parameters, manufacturing_cost_parameters, engineering_project_management_cost_parameters, imputed_cost_parameters, installation_erection_cost_parameters, other_cost_parameters, overhead_cost_parameters]

# Iterate over the search parameters in the set for each year contained in the list 
for year in parameter_column2:

    result_sheet.cell(row=1, column=col_index).value = year     # Set the year in row 1
    end_result_sheet.cell(row=1, column=col_index).value = year     # Set the year in row 1

# Reset cost centre variables to zero at the beginning of each year
    material_costs = 0
    manufacturing_costs = 0
    engineering_project_management_costs = 0
    imputed_costs = 0
    installation_erection_costs = 0
    other_costs = 0
    overhead_costs = 0

    for i, parameter in enumerate(parameter_column1, start=2):
        sum_parameter = 0
        
        for cell_1, cell_2, cell_3 in zip(source_sheet['B'][1:], source_sheet['E'][1:], source_sheet['X'][1:]): # iterate over 3 columns simultanuously
            if cell_1.value == parameter and cell_2.value == year:
                if cell_3.value is not None:
                    sum_parameter += cell_3.value

                    print(parameter, cell_1.value, cell_2.value, cell_3.value)


        result_sheet.cell(row=i, column=col_index).value = parameter
        result_sheet.cell(row=i, column=col_index + 1).value = sum_parameter

# Calculate cost centres based on parameter matching
        for j, cost_param_list in enumerate(cost_parameters, start=1):

            # If the parameter in the text matches the parameter in our list the costs are allocated to the coresponding cost centre
            if parameter in cost_param_list:
                if j == 1:
                    material_costs += sum_parameter
                elif j == 2:
                    manufacturing_costs += sum_parameter
                elif j == 3: 
                    engineering_project_management_costs += sum_parameter
                elif j == 4:
                    imputed_costs += sum_parameter
                elif j == 5:
                    installation_erection_costs += sum_parameter
                elif j == 6:
                    other_costs += sum_parameter
                elif j == 7:
                    overhead_costs += sum_parameter

                print(material_costs)
                print(manufacturing_costs)
                print(engineering_project_management_costs)
                print(installation_erection_costs)
                print(other_costs)
                print(overhead_costs)

# Add the values to the end result sheet
    end_result_sheet.cell(row=2, column=col_index).value = "Material costs"
    end_result_sheet.cell(row=2, column=col_index + 1).value = material_costs

    end_result_sheet.cell(row=3, column=col_index).value = "Manufacturing costs"
    end_result_sheet.cell(row=3, column=col_index + 1).value = manufacturing_costs

    end_result_sheet.cell(row=4, column=col_index).value = "Engineering and Project Management costs"
    end_result_sheet.cell(row=4, column=col_index + 1).value = engineering_project_management_costs

    end_result_sheet.cell(row=5, column=col_index).value = "Imputed costs"
    end_result_sheet.cell(row=5, column=col_index + 1).value = imputed_costs

    end_result_sheet.cell(row=6, column=col_index).value = "Installation and Erection costs"
    end_result_sheet.cell(row=6, column=col_index + 1).value = installation_erection_costs

    end_result_sheet.cell(row=7, column=col_index).value = "Other costs"
    end_result_sheet.cell(row=7, column=col_index + 1).value = other_costs

    end_result_sheet.cell(row=8, column=col_index).value = "Overhead costs"
    end_result_sheet.cell(row=8, column=col_index + 1).value = overhead_costs

# Increase the index so the next year can be added separately
    col_index += 2

output_filename = "output.xlsx"
output_path = os.path.join(os.getcwd(), output_filename)

workbook.save(output_path)

In [ ]:
print(material_costs)
print(manufacturing_costs)
print(engineering_project_management_costs)
print(imputed_costs)
print(installation_erection_costs)
print(other_costs)
print(overhead_costs)